In [1]:
# For plotting
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
# For conversion
from skimage.color import lab2rgb, rgb2lab, rgb2gray
from skimage import io
# For everything
import torch
import torch.nn as nn
import torch.nn.functional as F
# For our model
import torchvision
import torchvision.models as models
from torchvision import datasets, transforms
from torchmetrics import MeanSquaredError, PeakSignalNoiseRatio, StructuralSimilarityIndexMeasure
from PIL import Image
# For utilities
import os, shutil, time

In [2]:
%load_ext tensorboard
%tensorboard --logdir=runs

In [3]:
# colab i kaggle jeszcze nie testowane
colab = False
kaggle = False
test_number = '12_2'

In [4]:
color_imgs = 'outputs/color/'
gray_imgs = 'outputs/gray/'
checkpoints = 'checkpoints'
if colab:
    from google.colab import drive
    drive.mount('/content/drive')
    dataset = '/content/drive/MyDrive/MGU/cifar10/'
    
    color_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{color_imgs}'
    gray_imgs = f'/content/drive/MyDrive/MGU/{test_number}/{gray_imgs}'
    checkpoints = f'/content/drive/MyDrive/MGU/{test_number}/{checkpoints}'
elif kaggle:
    dataset = '/kaggle/input/cifar10/'
    
    color_imgs = f'{test_number}/{color_imgs}'
    gray_imgs = f'{test_number}/{gray_imgs}'
    checkpoints = f'{test_number}/{checkpoints}'
else:
    dataset = '../../datasets/cifar10/'

In [5]:
# Make folders and set parameters
os.makedirs(color_imgs, exist_ok=True)
os.makedirs(gray_imgs, exist_ok=True)
os.makedirs(checkpoints, exist_ok=True)
save_images = True
best_losses = [1e10, 1e10, 1e10]
best_epoch = -1
patience = 50
epochs = 500
batch_size = 128
SIZE = 32

In [6]:
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter()

In [7]:
# Check if GPU is available
use_gpu = torch.cuda.is_available()
print(use_gpu)

True


In [8]:
class LabImageFolder(torch.utils.data.Dataset):
    def __init__(self, paths, split='train'):
        if split == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC),
                transforms.RandomCrop(SIZE),
                transforms.RandomHorizontalFlip(),  
            ])
        elif split == 'val':
            self.transforms = transforms.Compose([
                transforms.Resize((SIZE, SIZE), transforms.InterpolationMode.BICUBIC), 
                transforms.RandomCrop(SIZE), 
            ])
            
        self.split = split
        self.size = SIZE
        self.paths = [os.path.join(paths, file) for file in os.listdir(paths) if os.path.isfile(
            os.path.join(paths, file))]
        
        
    def __getitem__(self, index):
        img = Image.open(self.paths[index]).convert("RGB")
        img_original = self.transforms(img)
        img_original = np.asarray(img_original)
        img_lab = rgb2lab(img_original)
        img_lab = (img_lab + 128) / 255
        img_ab = img_lab[:, :, 1:3]
        img_ab = torch.from_numpy(img_ab.transpose((2, 0, 1))).float()
        img_gray = rgb2gray(img_original)
        img_gray = torch.from_numpy(img_gray).unsqueeze(0).float()
        return img_gray, img_ab
    
    def __len__(self):
        return len(self.paths)

In [9]:
# Training
train_imagefolder = LabImageFolder(dataset + 'train')
train_loader = torch.utils.data.DataLoader(train_imagefolder, batch_size=batch_size, shuffle=True)
# Validation 
val_imagefolder = LabImageFolder(dataset + 'val' , 'val')
val_loader = torch.utils.data.DataLoader(val_imagefolder, batch_size=batch_size, shuffle=False)

In [10]:
kernel_size=3
stride_en=1
stride_de=1
padding=1
scale_factor=2
padding_mode='zeros'
channels_base = 16

class Autoencoder(nn.Module):
    def __init__(self, input_size=128):
        super(Autoencoder, self).__init__()

        self.conv1 = nn.Conv2d(1, channels_base, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)
        self.conv2 = nn.Conv2d(channels_base, channels_base * 2, kernel_size=kernel_size, stride=stride_en, padding=padding, padding_mode=padding_mode)

        self.convtrans1 = nn.ConvTranspose2d(channels_base * 2, channels_base, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans2 = nn.ConvTranspose2d(channels_base, channels_base // 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)
        self.convtrans3 = nn.ConvTranspose2d(channels_base // 2, 2, kernel_size=kernel_size, stride=stride_de, padding=padding, padding_mode=padding_mode)

        self.batchnorm1 = nn.BatchNorm2d(channels_base)
        self.batchnorm2 = nn.BatchNorm2d(channels_base * 2)
        self.batchnorm3 = nn.BatchNorm2d(channels_base)
        self.batchnorm4 = nn.BatchNorm2d(channels_base // 2)
        
    def forward(self, input):
        # encoder
        x = F.leaky_relu(self.batchnorm1(self.conv1(input)), negative_slope=0.1)
        x = F.avg_pool2d(x, kernel_size=3, stride=2, padding=1)
        x = F.leaky_relu(self.batchnorm2(self.conv2(x)), negative_slope=0.1)
        x = F.avg_pool2d(x, kernel_size=3, stride=2, padding=1)
        
        # decoder
        x = F.leaky_relu(self.batchnorm3(self.convtrans1(x)), negative_slope=0.1)
        x = F.interpolate(x, scale_factor=scale_factor)
        x = F.leaky_relu(self.batchnorm4(self.convtrans2(x)), negative_slope=0.1)
        x = F.interpolate(self.convtrans3(x), scale_factor=scale_factor)

        return x

In [11]:
model = Autoencoder()

In [12]:
criterion = [MeanSquaredError(), PeakSignalNoiseRatio(data_range=1.0), StructuralSimilarityIndexMeasure(data_range=1.0)]

In [13]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2)

In [14]:
# # Move model and loss function to GPU
if use_gpu: 
    criterion = [criterion[0].to("cuda"), criterion[1].to("cuda"), criterion[2].to("cuda")]
    model = model.cuda()

In [15]:
if use_gpu: 
    from torchsummary import summary
    summary(model, (1, SIZE, SIZE))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 32, 32]             160
       BatchNorm2d-2           [-1, 16, 32, 32]              32
            Conv2d-3           [-1, 32, 16, 16]           4,640
       BatchNorm2d-4           [-1, 32, 16, 16]              64
   ConvTranspose2d-5             [-1, 16, 8, 8]           4,624
       BatchNorm2d-6             [-1, 16, 8, 8]              32
   ConvTranspose2d-7            [-1, 8, 16, 16]           1,160
       BatchNorm2d-8            [-1, 8, 16, 16]              16
   ConvTranspose2d-9            [-1, 2, 16, 16]             146
Total params: 10,874
Trainable params: 10,874
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.43
Params size (MB): 0.04
Estimated Total Size (MB): 0.47
---------------------------------------------

In [16]:
class AverageMeter(object):
    '''A handy class from the PyTorch ImageNet tutorial''' 
    def __init__(self):
        self.reset()
    def reset(self):
        self.val, self.avg, self.sum, self.count = 0, 0, 0, 0
    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def to_rgb(grayscale_input, ab_input, save_path=None, save_name=None):
    '''Show/save rgb image from grayscale and ab channels
       Input save_path in the form {'grayscale': '/path/', 'colorized': '/path/'}'''
    plt.clf() # clear matplotlib 
    color_image = torch.cat((grayscale_input, ab_input), 0).numpy() # combine channels
    color_image = color_image.transpose((1, 2, 0))  # rescale for matplotlib
    color_image[:, :, 0:1] = color_image[:, :, 0:1] * 100
    color_image[:, :, 1:3] = color_image[:, :, 1:3] * 255 - 128   
    color_image = lab2rgb(color_image.astype(np.float64))
    grayscale_input = grayscale_input.squeeze().numpy()
    if save_path is not None and save_name is not None: 
        plt.imsave(arr=grayscale_input, fname='{}{}'.format(save_path['grayscale'], save_name), cmap='gray')
        plt.imsave(arr=color_image, fname='{}{}'.format(save_path['colorized'], save_name))

In [17]:
def validate(val_loader, model, criterion, save_images, epoch):
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]

    model.eval()
    already_saved_images = False
    for gray, ab in val_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()

        # Run model and record loss
        output_ab = model(gray) # throw away class predictions
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))

        # Save images to file
        if save_images and not already_saved_images:
            already_saved_images = True
            for j in range(min(len(output_ab), 10)): # save at most 5 images
                save_path = {'grayscale': gray_imgs, 'colorized': color_imgs}
                save_name = 'img-{}-epoch-{}.jpg'.format(j, epoch)
                to_rgb(gray[j].cpu(), ab_input=output_ab[j].detach().cpu(), save_path=save_path, save_name=save_name)

    print(f'Validate: MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print('Finished validation.')
    if epoch >= 0:
        writer.add_scalar("MSE/test", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/test", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/test", _loss[2].avg, epoch)
    return _loss[0].avg, _loss[1].avg, _loss[2].avg

In [18]:
def train(train_loader, model, criterion, optimizer, epoch):
    print(f'Starting training epoch {epoch}')
    _loss = [AverageMeter(), AverageMeter(), AverageMeter()]
    
    model.train()

    for gray, ab in train_loader:
        if use_gpu: 
            gray, ab = gray.cuda(), ab.cuda()
            
        optimizer.zero_grad()

        output_ab = model(gray) 
        loss = [criterion[0](output_ab, ab), criterion[1](output_ab, ab), criterion[2](output_ab, ab)]
        
        loss[0].backward()
        optimizer.step()
        
        _loss[0].update(loss[0].item(), gray.size(0))
        _loss[1].update(loss[1].item(), gray.size(0))
        _loss[2].update(loss[2].item(), gray.size(0))
        
    print(f'Epoch: {epoch}, MSE {_loss[0].val:.8f} ({_loss[0].avg:.8f}), PSNR {_loss[1].val:.8f} ({_loss[1].avg:.8f}), SSIM {_loss[2].val:.8f} ({_loss[2].avg:.8f})')

    print(f'Finished training epoch {epoch}')
    if epoch >= 0:
        writer.add_scalar("MSE/train", _loss[0].avg, epoch)
        writer.add_scalar("PSNR/train", _loss[1].avg, epoch)
        writer.add_scalar("SSIM/train", _loss[2].avg, epoch)

In [19]:
# Train model
for epoch in range(epochs):
    # Train for one epoch, then validate
    train(train_loader, model, criterion, optimizer, epoch)
    with torch.no_grad():
        losses = validate(val_loader, model, criterion, save_images, epoch)
    # Save checkpoint and replace old best model if current model is better
    if losses[0] < best_losses[0]:
        best_losses[0] = losses[0]
        best_epoch = epoch
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}.pth')
    if losses[1] < best_losses[1]:
        best_losses[1] = losses[1]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-PSNRLoss-{losses[1]:.8f}.pth')
    if losses[2] < best_losses[2]:
        best_losses[2] = losses[2]
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-SSIMLoss-{losses[2]:.8f}.pth')
    
    if epoch - best_epoch >= patience and epoch >= 100:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-MSELoss-{losses[0]:.8f}-early_stop.pth')
        break
    
    if epoch == epochs - 1:
        torch.save(model.state_dict(), f'{checkpoints}/epoch-{epoch}-last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')


Starting training epoch 0
Epoch: 0, MSE 0.00415553 (0.01136163), PSNR 23.81373215 (22.73930517), SSIM 0.66506022 (0.51631719)
Finished training epoch 0
Validate: MSE 0.00352527 (0.00352590), PSNR 24.52807045 (24.60812926), SSIM 0.57576752 (0.64997237)
Finished validation.
Starting training epoch 1
Epoch: 1, MSE 0.00323463 (0.00331804), PSNR 24.90175438 (24.82265656), SSIM 0.70519394 (0.68336603)
Finished training epoch 1
Validate: MSE 0.00345700 (0.00327088), PSNR 24.61300278 (24.94868554), SSIM 0.63192189 (0.70646879)
Finished validation.
Starting training epoch 2
Epoch: 2, MSE 0.00241908 (0.00305562), PSNR 26.16349220 (25.17984562), SSIM 0.75738192 (0.72228509)
Finished training epoch 2
Validate: MSE 0.00304934 (0.00291167), PSNR 25.15794373 (25.43557385), SSIM 0.66313601 (0.73309259)
Finished validation.
Starting training epoch 3
Epoch: 3, MSE 0.00273158 (0.00286482), PSNR 25.63585472 (25.45742380), SSIM 0.74472314 (0.73988810)
Finished training epoch 3
Validate: MSE 0.00311970 (0.0

Epoch: 30, MSE 0.00238626 (0.00235609), PSNR 26.22282028 (26.29882365), SSIM 0.76790965 (0.77800337)
Finished training epoch 30
Validate: MSE 0.00271028 (0.00245374), PSNR 25.66985703 (26.17197712), SSIM 0.71404076 (0.77251176)
Finished validation.
Starting training epoch 31
Epoch: 31, MSE 0.00231582 (0.00236172), PSNR 26.35294533 (26.29322809), SSIM 0.78245592 (0.77807591)
Finished training epoch 31
Validate: MSE 0.00266805 (0.00264911), PSNR 25.73806763 (25.88132634), SSIM 0.71696025 (0.77476486)
Finished validation.
Starting training epoch 32
Epoch: 32, MSE 0.00269761 (0.00235253), PSNR 25.69020271 (26.30660256), SSIM 0.76754415 (0.77814977)
Finished training epoch 32
Validate: MSE 0.00260315 (0.00244352), PSNR 25.84500504 (26.17601246), SSIM 0.71283531 (0.77523440)
Finished validation.
Starting training epoch 33
Epoch: 33, MSE 0.00232152 (0.00235681), PSNR 26.34226799 (26.29900395), SSIM 0.77505815 (0.77803370)
Finished training epoch 33
Validate: MSE 0.00265063 (0.00239087), PSNR 

Epoch: 60, MSE 0.00204541 (0.00231226), PSNR 26.89219666 (26.38060805), SSIM 0.77320445 (0.77873940)
Finished training epoch 60
Validate: MSE 0.00265055 (0.00280140), PSNR 25.76663208 (25.64669058), SSIM 0.71573532 (0.77274737)
Finished validation.
Starting training epoch 61
Epoch: 61, MSE 0.00271555 (0.00231105), PSNR 25.66142464 (26.38468296), SSIM 0.77448583 (0.77876960)
Finished training epoch 61
Validate: MSE 0.00250741 (0.00239779), PSNR 26.00774956 (26.25691145), SSIM 0.71649033 (0.77635756)
Finished validation.
Starting training epoch 62
Epoch: 62, MSE 0.00231657 (0.00230557), PSNR 26.35154915 (26.39511544), SSIM 0.76171952 (0.77887859)
Finished training epoch 62
Validate: MSE 0.00293201 (0.00260021), PSNR 25.32834435 (25.93478351), SSIM 0.71455979 (0.77585297)
Finished validation.
Starting training epoch 63
Epoch: 63, MSE 0.00219476 (0.00231427), PSNR 26.58613586 (26.37931425), SSIM 0.78812873 (0.77885321)
Finished training epoch 63
Validate: MSE 0.00295312 (0.00241129), PSNR 

Epoch: 90, MSE 0.00223186 (0.00228764), PSNR 26.51332092 (26.43039465), SSIM 0.77444255 (0.77924462)
Finished training epoch 90
Validate: MSE 0.00271359 (0.00245665), PSNR 25.66456413 (26.17868801), SSIM 0.71465153 (0.77769062)
Finished validation.
Starting training epoch 91
Epoch: 91, MSE 0.00219176 (0.00230119), PSNR 26.59206581 (26.40638565), SSIM 0.75774276 (0.77905516)
Finished training epoch 91
Validate: MSE 0.00248317 (0.00235106), PSNR 26.04993439 (26.35149528), SSIM 0.71883124 (0.77635850)
Finished validation.
Starting training epoch 92
Epoch: 92, MSE 0.00169302 (0.00228977), PSNR 27.71337128 (26.42500651), SSIM 0.81729662 (0.77914511)
Finished training epoch 92
Validate: MSE 0.00270752 (0.00234452), PSNR 25.67428398 (26.36149448), SSIM 0.71440041 (0.77909776)
Finished validation.
Starting training epoch 93
Epoch: 93, MSE 0.00219445 (0.00228917), PSNR 26.58675194 (26.42540510), SSIM 0.78579664 (0.77932436)
Finished training epoch 93
Validate: MSE 0.00275495 (0.00240381), PSNR 

Epoch: 120, MSE 0.00176652 (0.00227884), PSNR 27.52881432 (26.44874958), SSIM 0.79893047 (0.77925000)
Finished training epoch 120
Validate: MSE 0.00281221 (0.00277954), PSNR 25.50952339 (25.68982893), SSIM 0.71339923 (0.77275567)
Finished validation.
Starting training epoch 121
Epoch: 121, MSE 0.00288501 (0.00228699), PSNR 25.39851952 (26.42763427), SSIM 0.77058327 (0.77926830)
Finished training epoch 121
Validate: MSE 0.00277841 (0.00228695), PSNR 25.56203461 (26.47433241), SSIM 0.71438336 (0.77980148)
Finished validation.
Starting training epoch 122
Epoch: 122, MSE 0.00198003 (0.00227863), PSNR 27.03327751 (26.44508096), SSIM 0.79877037 (0.77925462)
Finished training epoch 122
Validate: MSE 0.00294719 (0.00243031), PSNR 25.30591583 (26.21257487), SSIM 0.71452951 (0.77733727)
Finished validation.
Starting training epoch 123
Epoch: 123, MSE 0.00206807 (0.00228213), PSNR 26.84434891 (26.44035922), SSIM 0.78122491 (0.77927500)
Finished training epoch 123
Validate: MSE 0.00269470 (0.00283

Validate: MSE 0.00262379 (0.00244292), PSNR 25.81070328 (26.18973961), SSIM 0.71750009 (0.77246803)
Finished validation.
Starting training epoch 150
Epoch: 150, MSE 0.00271650 (0.00227657), PSNR 25.65989304 (26.45052677), SSIM 0.75943244 (0.77950762)
Finished training epoch 150
Validate: MSE 0.00265068 (0.00250073), PSNR 25.76642036 (26.09196700), SSIM 0.71573281 (0.77412154)
Finished validation.
Starting training epoch 151
Epoch: 151, MSE 0.00251289 (0.00226786), PSNR 25.99826622 (26.46388696), SSIM 0.75724781 (0.77933236)
Finished training epoch 151
Validate: MSE 0.00262895 (0.00243227), PSNR 25.80217552 (26.21702269), SSIM 0.71933568 (0.78077253)
Finished validation.
Starting training epoch 152
Epoch: 152, MSE 0.00174908 (0.00227445), PSNR 27.57189751 (26.45495111), SSIM 0.81894934 (0.77938276)
Finished training epoch 152
Validate: MSE 0.00270622 (0.00226247), PSNR 25.67636108 (26.51374724), SSIM 0.71642113 (0.77767085)
Finished validation.
Starting training epoch 153
Epoch: 153, MS

Epoch: 179, MSE 0.00297483 (0.00226905), PSNR 25.26537514 (26.46372566), SSIM 0.74565816 (0.77950378)
Finished training epoch 179
Validate: MSE 0.00262622 (0.00226933), PSNR 25.80669403 (26.50309852), SSIM 0.71880448 (0.77772178)
Finished validation.
Starting training epoch 180
Epoch: 180, MSE 0.00224618 (0.00227550), PSNR 26.48554802 (26.45115292), SSIM 0.79170614 (0.77926229)
Finished training epoch 180
Validate: MSE 0.00275892 (0.00280137), PSNR 25.59260750 (25.67116966), SSIM 0.71278369 (0.76964171)
Finished validation.
Starting training epoch 181
Epoch: 181, MSE 0.00208794 (0.00227003), PSNR 26.80281258 (26.46288062), SSIM 0.77177423 (0.77943654)
Finished training epoch 181
Validate: MSE 0.00264833 (0.00231881), PSNR 25.77028656 (26.41427223), SSIM 0.71476936 (0.78000710)
Finished validation.
Starting training epoch 182
Epoch: 182, MSE 0.00270338 (0.00226798), PSNR 25.68092537 (26.46710306), SSIM 0.74834687 (0.77943366)
Finished training epoch 182
Validate: MSE 0.00264948 (0.00247

<Figure size 432x288 with 0 Axes>

In [20]:
torch.save(model.state_dict(), f'{checkpoints}/last-{losses[0]:.8f}-{losses[1]:.8f}-{losses[2]:.8f}.pth')

In [21]:
# Validate
save_images = True
with torch.no_grad():
    validate(val_loader, model, criterion, save_images, -1)

Validate: MSE 0.00312323 (0.00246099), PSNR 25.05395699 (26.15515598), SSIM 0.71056938 (0.77324800)
Finished validation.


<Figure size 432x288 with 0 Axes>

In [22]:
# # Show images 
# image_pairs = []

# for i in range(10):
#     image_pairs.append((f'{color_imgs}img-{i}-epoch-{best_epoch}.jpg', f'{gray_imgs}img-{i}-epoch-{best_epoch}.jpg'))
    
# for c, g in image_pairs:
#   color = mpimg.imread(c)
#   gray  = mpimg.imread(g)
#   f, axarr = plt.subplots(1, 2)
#   f.set_size_inches(15, 15)
#   axarr[0].imshow(gray, cmap='gray')
#   axarr[1].imshow(color)
#   axarr[0].axis('off'), axarr[1].axis('off')
#   plt.show()